In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: review
Azure region: eastus2
Subscription id: d6ebc258-732e-4a76-8d46-be5af7f5737d
Resource group: beginner


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "cpu-cluster1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C':choice(0.01, 0.1, 1),
        '--max_iter':choice(25, 60, 130, 200, 320)
    }
) 

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1, delay_evaluation=4)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    arguments=['--C', 5,'--max_iter', 1000],
    compute_target=aml_compute,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=18,
    max_concurrent_runs=4
)

In [6]:
from azureml.widgets import RunDetails
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
run=exp.submit(hyperdrive_config)
RunDetails(run).show()
run.get_status()
run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_2d2a2f3c-086f-449f-92e9-f9bafae35009
Web View: https://ml.azure.com/runs/HD_2d2a2f3c-086f-449f-92e9-f9bafae35009?wsid=/subscriptions/d6ebc258-732e-4a76-8d46-be5af7f5737d/resourcegroups/beginner/workspaces/review&tid=52299ebb-8afb-45a8-8e21-7df88891d90a

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-02-20T21:09:18.667148][API][INFO]Experiment created<END>\n""<START>[2022-02-20T21:09:19.389184][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-02-20T21:09:19.935945][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_2d2a2f3c-086f-449f-92e9-f9bafae35009
Web View: https://ml.azure.com/runs/HD_2d2a2f3c-086f-449f-92e9-f9bafae35009?wsid=/subscriptions/d6ebc258-732e-4a76-8d46-be5af7f5737d/resourcegroups/beginner/workspaces/review&tid=52299ebb-8afb-45a8-8e21-7df88891d90a



{'runId': 'HD_2d2a2f3c-086f-449f-92e9-f9bafae35009',
 'target': 'cpu-cluster1',
 'status': 'Completed',
 'startTimeUtc': '2022-02-20T21:09:18.470638Z',
 'endTimeUtc': '2022-02-20T21:33:59.244444Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'e84c8802-98cd-4dc2-bcc1-43b02603d75d',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1064-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.37.0',
  'space_size': '15',
  'score': '0.9083459787556905',
  'best_child_run_id': 'HD_2d2a2f3c-086f-449f-92e9-f9bafae35009_11',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://review1864130175.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_2d2a2f3c-086f-449f-92e9-f9bafae35009/azu

2022-02-20:22:02:10,755 INFO     [explanation_client.py:332] Using default datastore for uploads


In [7]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = run.get_best_run_by_primary_metric()

#print('best run details:', best_run.get_details())
print('best run file names:', best_run.get_file_names())
print('best run metrics:', best_run.get_metrics())
model = best_run.register_model(model_name = 'sklearn_log_reg_train_P', model_path ='' )
print(model)

best run file names: ['logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']
best run metrics: {'Regularization Strength:': 0.1, 'Max iterations:': 320, 'Accuracy': 0.9083459787556905}
Model(workspace=Workspace.create(name='review', subscription_id='d6ebc258-732e-4a76-8d46-be5af7f5737d', resource_group='beginner'), name=sklearn_log_reg_train_P, id=sklearn_log_reg_train_P:2, version=2, tags={}, properties={})


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")
ds.to_pandas_dataframe().head()


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [9]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
### YOUR DATA OBJECT HERE ###
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=280)
train_dt = pd.concat([x_train, y_train], axis=1)

In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_dt,
    label_column_name='y',
    n_cross_validations=6,
    debug_log = 'automl_errors.log'
    )

In [11]:
from azureml.widgets import RunDetails
# Submit your automl run

### YOUR CODE HERE ###
auto_run = exp.submit(config = automl_config, show_output = True)
RunDetails(auto_run).show()
auto_run.wait_for_completion(show_output=False)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


2022-02-20:21:36:05,70 INFO     [modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-02-20:21:36:05,90 INFO     [modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-02-20:21:36:29,152 INFO     [utils.py:159] NumExpr defaulting to 4 threads.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_5a867da0-5fdd-433b-b080-4bdcc05baee1,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2022-02-20:22:01:01,516 INFO     [explanation_client.py:332] Using default datastore for uploads


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_5a867da0-5fdd-433b-b080-4bdcc05baee1',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-02-20T21:36:30.523663Z',
 'endTimeUtc': '2022-02-20T21:59:34.595401Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '6',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"d6ebc258-732e-4a76-8d46-be5af7f5737d","resource_group":"beginner","workspace_name":"review","region":"eastus2","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNot

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

2022-02-20:22:02:16,510 INFO     [explanation_client.py:332] Using default datastore for uploads
2022-02-20:22:02:22,359 INFO     [explanation_client.py:332] Using default datastore for uploads
2022-02-20:22:02:27,856 INFO     [explanation_client.py:332] Using default datastore for uploads
2022-02-20:22:02:33,158 INFO     [explanation_client.py:332] Using default datastore for uploads
2022-02-20:22:02:39,42 INFO     [explanation_client.py:332] Using default datastore for uploads
2022-02-20:22:02:46,338 INFO     [explanation_client.py:332] Using default datastore for uploads
2022-02-20:22:02:52,257 INFO     [explanation_client.py:332] Using default datastore for uploads
2022-02-20:22:02:58,134 INFO     [explanation_client.py:332] Using default datastore for uploads
2022-02-20:22:03:04,240 INFO     [explanation_client.py:332] Using default datastore for uploads
2022-02-20:22:03:10,495 INFO     [explanation_client.py:332] Using default datastore for uploads
2022-02-20:22:03:22,920 INFO   

In [12]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_best_run, automl_fitted_model = auto_run.get_output()
automl_best_run.get_metrics()

{'precision_score_macro': 0.7989632633067822,
 'f1_score_macro': 0.7825696460989234,
 'accuracy': 0.9175373935455005,
 'norm_macro_recall': 0.5381645523837337,
 'average_precision_score_macro': 0.8243439212918012,
 'balanced_accuracy': 0.7690822761918669,
 'AUC_weighted': 0.9469297038392153,
 'AUC_macro': 0.9469297038392154,
 'AUC_micro': 0.9804031200704246,
 'f1_score_weighted': 0.915301336061893,
 'weighted_accuracy': 0.954595347724818,
 'recall_score_weighted': 0.9175373935455005,
 'matthews_correlation': 0.5670135308993062,
 'recall_score_macro': 0.7690822761918669,
 'recall_score_micro': 0.9175373935455005,
 'f1_score_micro': 0.9175373935455005,
 'precision_score_micro': 0.9175373935455005,
 'log_loss': 0.2219629518952592,
 'average_precision_score_micro': 0.9812283779258718,
 'average_precision_score_weighted': 0.955039650529146,
 'precision_score_weighted': 0.9138090462420522,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_5a867da0-5fdd-433b-b080-4bdcc05baee1_30/

2022-02-20:22:04:54,411 INFO     [explanation_client.py:332] Using default datastore for uploads
2022-02-20:22:05:02,381 INFO     [explanation_client.py:332] Using default datastore for uploads
2022-02-20:22:05:10,425 INFO     [explanation_client.py:332] Using default datastore for uploads
